In [ ]:
!pip install pytesseract
!pip install pdf2image

!apt-get install poppler-utils
!sudo apt-get install tesseract-ocr-fas

In [ ]:
!wget https://github.com/tesseract-ocr/tessdata/raw/main/fas.traineddata

--2024-01-14 06:26:04--  https://github.com/tesseract-ocr/tessdata/raw/main/fas.traineddata
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/fas.traineddata [following]
--2024-01-14 06:26:04--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/fas.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 561272 (548K) [application/octet-stream]
Saving to: ‘fas.traineddata’

fas.traineddata     100%[===================>] 548.12K  --.-KB/s    in 0.02s   

2024-01-14 06:26:04 (35.0 MB/s) - ‘fas.traineddata’ saved [561272/561272]



In [ ]:
import os
import csv
import pytesseract
from pdf2image import convert_from_path
import shutil
import gdown
import gc

* Path to the Tesseract executable (modify this based on your installation)

In [ ]:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

* OCR

In [ ]:
def extract_text_from_image(img, lang='fas'):
    return pytesseract.image_to_string(img, lang=lang)

* PDF to Text

In [ ]:
def extract_text_from_pdf(pdf_name):
    # extracted_data = []
    images = convert_from_path(f'pdfs/{pdf_name}.pdf',first_page=2, grayscale=True)
    with open(f'texts/{pdf_name}.txt', 'w', encoding='utf-8', newline='') as text_file:
        for i, img in enumerate(images):
            progress = int((i+1)/len(images)*20)
            print(f"\r[{''.join(['=' if j<progress else '>' if j==progress else '-' for j in range(1,21)])}] {i+1:2d}/{len(images)}",end='')
            image_filename = f"/content/images/image_{i + 1}.png"
            img.save(image_filename, 'PNG')
            extracted_text = extract_text_from_image(image_filename)
            text_file.write(f"{extracted_text}\n\n")
            # extracted_data.append({
            #         "Page": i + 1,
            #         "Text": extracted_text
                # })
    print(" \u2713")
    # shutil.move(f'/content/{pdf_name}.txt', f'./texts/{pdf_name}.txt')
    shutil.move(f'./pdfs/{pdf_name}.pdf', f'./pdfs_done/{pdf_name}.pdf')
    # Cleanup memory
    del(images)
    gc.collect()
    # return extracted_data

* Save dataset to CSV

In [ ]:
def save_to_csv(data, csv_name):
    # Specify the CSV file path
    csv_columns = ["Page", "Text"]

    # Write the data to the CSV file
    with open(f'dataset/{csv_name}.csv', 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [ ]:
def save_to_text(file_name,data):
    with open(f'texts/{file_name}.txt', 'w', encoding='utf-8') as file_name:
        # Process each image and extract text
        for extracted_text in data:
            # Save extracted text to the text file
            file_name.write(f"{extracted_text}\n\n")

* Create textual dataset from pdfs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/text_dataset/pdfs')
names = [name for name in os.listdir('.') if name.endswith('.PDF')]
print(len(names))
for name in names:
    os.rename(name,f'{name[:-4]}.pdf')

In [ ]:
os.makedirs('/content/images')
os.chdir('/content/drive/MyDrive/text_dataset')

In [ ]:
num_pdfs = len(os.listdir('./pdfs'))
for i, file_name in enumerate(os.listdir('./pdfs')):
    print(f'[{i+1}/{num_pdfs}]',file_name[:-4])
    extract_text_from_pdf(file_name[:-4])